In [1]:
from shapely.wkt import loads
geom = loads("POLYGON((-1.26 51.75, -1.25 51.75, -1.25 51.76, -1.26 51.76, -1.26 51.75))")


In [2]:
import geopandas as gpd
buffer_m=1000
gdf_latlon = gpd.GeoDataFrame(geometry=[geom], crs="EPSG:4326")
print(gdf_latlon.iloc[0].geometry)
gdf_ukgrid=gdf_latlon.to_crs(27700)
print(gdf_ukgrid.iloc[0].geometry)


POLYGON ((-1.26 51.75, -1.25 51.75, -1.25 51.76, -1.26 51.76, -1.26 51.75))
POLYGON ((451181.8067423534 205959.42761765362, 451872.1365434999 205966.47458434716, 451860.70613464457 207078.6670373804, 451170.5288039437 207071.62065761362, 451181.8067423534 205959.42761765362))


In [6]:
gdf_ukgrid["geometry"] = gdf_ukgrid.buffer(buffer_m,resolution=4, join_style="mitre")
print(gdf_ukgrid.iloc[0].geometry)


POLYGON ((449202.1904339386 203939.11526180914, 449150.35352661525 209051.0998678343, 453840.0482665709 209098.97940351022, 453892.58599731687 203986.99536384118, 449202.1904339386 203939.11526180914))


In [7]:
gdf_buffered_latlon = gdf_ukgrid.to_crs(4326)
wkt_with_buffer = gdf_buffered_latlon.iloc[0].geometry
print(wkt_with_buffer)


POLYGON ((-1.2889586296821363 51.73201354199302, -1.2889880438846308 51.777976757023254, -1.2210119164828335 51.777976650545504, -1.2210413306757637 51.7320136485258, -1.2889586296821363 51.73201354199302))


In [9]:
gdf_buffered_latlon.explore()

These are teh recognisable lists we are interested in for the MVP

Protected lists 
WCA Schedule 5  (dr2402)
Schedule 1 (Birds)  (dr2413)
Schedule 8 (Plants) (dr2414)

Priority lists
NERC Section 41 (England) (dr590)
UK BAP Priority Species (dr583)


Now get the species counts for all these lists


In [30]:
import requests
QUERY_A_TOP_SPECIES_COUNTS_BASE_URL = 'https://records-ws.nbnatlas.org/occurrence/facets?facets=taxon_name&flimit=2&q=((species_list_uid%3A"dr2402" OR species_list_uid%3A"dr2413" OR species_list_uid%3A"dr2414" OR species_list_uid%3A"dr590" OR species_list_uid%3A"dr583") AND -(occurrence_status%3A"absent" OR identification_verification_status%3A"Unconfirmed" OR identification_verification_status%3A"Unconfirmed - not reviewed" OR identification_verification_status%3A"Unconfirmed - plausible"))'
url = QUERY_A_TOP_SPECIES_COUNTS_BASE_URL + '&wkt=POLYGON((-1.26 51.75, -1.25 51.75, -1.25 51.76, -1.26 51.76, -1.26 51.75))'
print(url)



https://records-ws.nbnatlas.org/occurrence/facets?facets=taxon_name&flimit=0&q=((species_list_uid%3A"dr2402" OR species_list_uid%3A"dr2413" OR species_list_uid%3A"dr2414" OR species_list_uid%3A"dr590" OR species_list_uid%3A"dr583") AND -(occurrence_status%3A"absent" OR identification_verification_status%3A"Unconfirmed" OR identification_verification_status%3A"Unconfirmed - not reviewed" OR identification_verification_status%3A"Unconfirmed - plausible"))&wkt=POLYGON((-1.26 51.75, -1.25 51.75, -1.25 51.76, -1.26 51.76, -1.26 51.75))


In [31]:
response = requests.get(url)
data = response.json()

for fieldResult in data[0]["fieldResult"]:
    print(f'{fieldResult["label"]} {fieldResult["count"]}')

In [32]:
print(data)

[{'fieldName': 'taxon_name', 'fieldResult': [], 'count': 45}]


In [33]:
QUERY_A2_UNIQUE_SPECIES_COUNT_BASE_URL = 'https://records-ws.nbnatlas.org/occurrence/facets?facets=taxon_name&flimit=0&q=((species_list_uid%3A"dr2402" OR species_list_uid%3A"dr2413" OR species_list_uid%3A"dr2414" OR species_list_uid%3A"dr590" OR species_list_uid%3A"dr583") AND -(occurrence_status%3A"absent" OR identification_verification_status%3A"Unconfirmed" OR identification_verification_status%3A"Unconfirmed - not reviewed" OR identification_verification_status%3A"Unconfirmed - plausible"))'
url = QUERY_A2_UNIQUE_SPECIES_COUNT_BASE_URL+f'&species_list_uid=dr2402&wkt={wkt_with_buffer}'
response = requests.get(url)

data = response.json()
print(data)

[{'fieldName': 'taxon_name', 'fieldResult': [], 'count': 45}]


In [34]:
tmp_dict = {'firstname':'john', 'lastname':'jones'}
new_dict = {**tmp_dict,'age':21}
print(new_dict)

{'firstname': 'john', 'lastname': 'jones', 'age': 21}
